In [ ]:
# Install required libraries
!pip install -q xgboost catboost lightgbm imbalanced-learn tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.3 MB/s eta 0:00:00


In [ ]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
# ==================== LOAD AND EXPLORE DATA ====================
train_data = pd.read_csv('train.csv')

print(f"Original data shape: {train_data.shape}")
print(f"\nDisorder Subclass distribution:")
print(train_data['Disorder Subclass'].value_counts())
print(f"\nClass balance ratio: {train_data['Disorder Subclass'].value_counts().min() / train_data['Disorder Subclass'].value_counts().max():.3f}")

Original data shape: (22083, 45)

Disorder Subclass distribution:
Disorder Subclass
Leigh syndrome                         5160
Mitochondrial myopathy                 4405
Cystic fibrosis                        3448
Tay-Sachs                              2833
Diabetes                               1817
Hemochromatosis                        1355
Leber's hereditary optic neuropathy     648
Alzheimer's                             152
Cancer                                   97
Name: count, dtype: int64

Class balance ratio: 0.019


In [ ]:
# ==================== DATA PREPROCESSING ====================
print("\n" + "="*60)
print("DATA PREPROCESSING")
print("="*60)

# Dropping irrelevant columns
drop_columns = ['Patient Id', 'Patient First Name', 'Family Name', "Father's name",
                "Institute Name", "Location of Institute", "Status", "Parental consent", "Genetic Disorder"]
train_data = train_data.drop(columns=drop_columns, errors='ignore')

# Keep Disorder Subclass as target
train_data = train_data.dropna(subset=['Disorder Subclass'])


DATA PREPROCESSING


In [ ]:
# ==================== ADVANCED FEATURE ENGINEERING ====================

# Replacing missing values in numeric columns with median (BETTER than mean for skewed data)
numeric_columns = ['No. of previous abortion', 'White Blood cell count (thousand per microliter)',
                   'Patient Age', 'Blood cell count (mcL)', "Mother's age", "Father's age"]
for col in numeric_columns:
    if col in train_data.columns:
        train_data[col] = train_data[col].fillna(train_data[col].median())
        train_data[col] = train_data[col].astype('float64')

# Create derived features from existing ones
if 'Patient Age' in train_data.columns:
    train_data['Age_squared'] = train_data['Patient Age'] ** 2
    train_data['Age_log'] = np.log1p(train_data['Patient Age'])

if "Mother's age" in train_data.columns and "Father's age" in train_data.columns:
    train_data['Parental_age_diff'] = abs(train_data["Mother's age"] - train_data["Father's age"])
    train_data['Parental_age_sum'] = train_data["Mother's age"] + train_data["Father's age"]
    train_data['Parental_age_avg'] = (train_data["Mother's age"] + train_data["Father's age"]) / 2

# Sum of symptoms (important feature)
symptom_cols = ['Symptom 1', 'Symptom 2', 'Symptom 3', 'Symptom 4', 'Symptom 5']
symptom_cols = [col for col in symptom_cols if col in train_data.columns]
if symptom_cols:
    for col in symptom_cols:
        train_data[col] = train_data[col].fillna(0).astype('int64')
    train_data['Total_symptoms'] = train_data[symptom_cols].sum(axis=1)
    train_data['Symptoms_ratio'] = train_data['Total_symptoms'] / len(symptom_cols)

# Blood cell ratio (important clinical feature)
if 'Blood cell count (mcL)' in train_data.columns and 'White Blood cell count (thousand per microliter)' in train_data.columns:
    train_data['WBC_RBC_ratio'] = train_data['White Blood cell count (thousand per microliter)'] / (train_data['Blood cell count (mcL)'] + 1e-6)

# Replacing missing values in categorical columns with 'Unknown'
categorical_columns = ['Blood test result', 'Birth defects', 'Gender',
                       'Heart Rate (rates/min', 'Respiratory Rate (breaths/min)',
                       'Follow-up', 'Place of birth']
categorical_columns = [col for col in categorical_columns if col in train_data.columns]
train_data[categorical_columns] = train_data[categorical_columns].fillna('Unknown')

# Replacing various 'NA' and similar values with 'No'
train_data = train_data.replace(["-", "Not applicable", "Not available", "None", "No record"], "No")

# Converting binary categorical variables to 0 and 1
binary_var = ["Maternal gene", "Genes in mother's side", 'Inherited from father', 'Paternal gene',
              'Birth asphyxia', 'Folic acid details (peri-conceptional)',
              'H/O serious maternal illness', 'H/O radiation exposure (x-ray)',
              'H/O substance abuse', 'Assisted conception IVF/ART',
              'History of anomalies in previous pregnancies',
              'Autopsy shows birth defect (if applicable)']
binary_var = [col for col in binary_var if col in train_data.columns]
for col in binary_var:
    train_data[col] = train_data[col].replace({"Yes": 1, "No": 0})
    train_data[col] = train_data[col].fillna(0).astype('int64')

# Create binary inheritance pattern feature
genetic_features = [col for col in binary_var if 'gene' in col.lower() or 'inherited' in col.lower()]
if genetic_features:
    train_data['Genetic_inheritance_score'] = train_data[genetic_features].sum(axis=1)

# Encode categorical columns
label_encoders = {}
for col in categorical_columns:
    if col in train_data.columns:
        le = LabelEncoder()
        train_data[col] = le.fit_transform(train_data[col].astype(str))
        label_encoders[col] = le

# Target encoding
label_encoder_target = LabelEncoder()
y_encoded = label_encoder_target.fit_transform(train_data['Disorder Subclass'])
num_classes = len(label_encoder_target.classes_)

print(f"\nNumber of classes: {num_classes}")
print(f"Classes: {label_encoder_target.classes_}")

# Features
X = train_data.drop('Disorder Subclass', axis=1)
print(f"\nFinal feature shape: {X.shape}")



Number of classes: 9
Classes: ["Alzheimer's" 'Cancer' 'Cystic fibrosis' 'Diabetes' 'Hemochromatosis'
 "Leber's hereditary optic neuropathy" 'Leigh syndrome'
 'Mitochondrial myopathy' 'Tay-Sachs']

Final feature shape: (19915, 44)


In [ ]:
print("\n" + "="*60)
print("HANDLING CLASS IMBALANCE WITH SMOTE")
print("="*60)

# Impute any remaining NaNs in X before SMOTE
# Using median imputation as a robust choice for numerical features
X = X.fillna(X.median())

# Use SMOTE to handle imbalanced classes
smote = SMOTE(random_state=42, k_neighbors=5)
X_balanced, y_balanced = smote.fit_resample(X, y_encoded)

print(f"\nBalanced data shape: {X_balanced.shape}")
print(f"Balanced class distribution:")
unique, counts = np.unique(y_balanced, return_counts=True)
for u, c in zip(unique, counts):
    print(f"  Class {u}: {c} samples")

# Split with balanced data
X_train, X_test, y_train, y_test = train_test_split(
    X_balanced, y_balanced, test_size=0.2, random_state=42, stratify=y_balanced
)

print(f"\nTrain set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")


HANDLING CLASS IMBALANCE WITH SMOTE

Balanced data shape: (46440, 44)
Balanced class distribution:
  Class 0: 5160 samples
  Class 1: 5160 samples
  Class 2: 5160 samples
  Class 3: 5160 samples
  Class 4: 5160 samples
  Class 5: 5160 samples
  Class 6: 5160 samples
  Class 7: 5160 samples
  Class 8: 5160 samples

Train set size: (37152, 44)
Test set size: (9288, 44)


In [ ]:
# ==================== FEATURE SCALING ====================
# Use RobustScaler for better handling of outliers
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# ==================== MODEL 1: OPTIMIZED XGBOOST ====================
print("\n" + "="*60)
print("TRAINING OPTIMIZED XGBOOST")
print("="*60)

xgb_model = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=7,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    colsample_bylevel=0.8,
    gamma=1.0,
    min_child_weight=1,
    objective='multi:softmax',
    num_class=num_classes,
    random_state=42,
    tree_method='hist',
    eval_metric='mlogloss',
    verbosity=0,
    n_jobs=-1
)

xgb_model.fit(X_train_scaled, y_train, verbose=False)
y_pred_xgb = xgb_model.predict(X_test_scaled)
xgb_acc = accuracy_score(y_test, y_pred_xgb)
xgb_f1 = f1_score(y_test, y_pred_xgb, average='weighted')

print(f"XGBoost Accuracy: {xgb_acc:.4f}")
print(f"XGBoost F1-Score (weighted): {xgb_f1:.4f}")


TRAINING OPTIMIZED XGBOOST
XGBoost Accuracy: 0.6693
XGBoost F1-Score (weighted): 0.6669


In [ ]:
# ==================== MODEL 2: LIGHTGBM ====================
print("\n" + "="*60)
print("TRAINING LIGHTGBM")
print("="*60)

lgb_model = lgb.LGBMClassifier(
    n_estimators=500,
    max_depth=7,
    learning_rate=0.05,
    num_leaves=31,
    subsample=0.9,
    colsample_bytree=0.9,
    min_child_samples=20,
    reg_alpha=0.1,
    reg_lambda=0.1,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

lgb_model.fit(X_train_scaled, y_train)
y_pred_lgb = lgb_model.predict(X_test_scaled)
lgb_acc = accuracy_score(y_test, y_pred_lgb)
lgb_f1 = f1_score(y_test, y_pred_lgb, average='weighted')

print(f"LightGBM Accuracy: {lgb_acc:.4f}")
print(f"LightGBM F1-Score (weighted): {lgb_f1:.4f}")


TRAINING LIGHTGBM
LightGBM Accuracy: 0.6844
LightGBM F1-Score (weighted): 0.6837


In [ ]:
# ==================== MODEL 3: CATBOOST ====================
print("\n" + "="*60)
print("TRAINING CATBOOST")
print("="*60)

cat_model = CatBoostClassifier(
    iterations=500,
    depth=7,
    learning_rate=0.05,
    subsample=0.9,
    bootstrap_type='Bernoulli', # Changed to Bernoulli
    random_state=42,
    verbose=False,
    task_type='CPU'
)

cat_model.fit(X_train_scaled, y_train, verbose=False)
y_pred_cat = cat_model.predict(X_test_scaled)
cat_acc = accuracy_score(y_test, y_pred_cat)
cat_f1 = f1_score(y_test, y_pred_cat, average='weighted')

print(f"CatBoost Accuracy: {cat_acc:.4f}")
print(f"CatBoost F1-Score (weighted): {cat_f1:.4f}")


TRAINING CATBOOST
CatBoost Accuracy: 0.6501
CatBoost F1-Score (weighted): 0.6466


In [ ]:
# ==================== MODEL 4: RANDOM FOREST ====================
print("\n" + "="*60)
print("TRAINING RANDOM FOREST")
print("="*60)

rf_model = RandomForestClassifier(
    n_estimators=500,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)

rf_model.fit(X_train_scaled, y_train)
y_pred_rf = rf_model.predict(X_test_scaled)
rf_acc = accuracy_score(y_test, y_pred_rf)
rf_f1 = f1_score(y_test, y_pred_rf, average='weighted')

print(f"Random Forest Accuracy: {rf_acc:.4f}")
print(f"Random Forest F1-Score (weighted): {rf_f1:.4f}")


TRAINING RANDOM FOREST
Random Forest Accuracy: 0.6760
Random Forest F1-Score (weighted): 0.6717


In [ ]:
# ==================== MODEL 5: NEURAL NETWORK (IMPROVED) ====================
print("\n" + "="*60)
print("TRAINING IMPROVED NEURAL NETWORK")
print("="*60)

y_train_onehot = keras.utils.to_categorical(y_train, num_classes)
y_test_onehot = keras.utils.to_categorical(y_test, num_classes)

nn_model = keras.Sequential([
    layers.Input(shape=(X_train_scaled.shape[1],)),

    # Batch norm first
    layers.BatchNormalization(),

    # Dense layers with decreasing units
    layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.4),

    layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0005)),
    layers.BatchNormalization(),
    layers.Dropout(0.2),

    layers.Dense(32, activation='relu'),

    layers.Dense(num_classes, activation='softmax')
])

nn_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\nNeural Network Architecture:")
nn_model.summary()

early_stopping = EarlyStopping(
    monitor='val_loss', patience=20, restore_best_weights=True, verbose=0
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=8, min_lr=1e-7, verbose=0
)

print("\nTraining Neural Network...")
history = nn_model.fit(
    X_train_scaled, y_train_onehot,
    validation_data=(X_test_scaled, y_test_onehot),
    epochs=150,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=0
)

y_pred_nn_prob = nn_model.predict(X_test_scaled, verbose=0)
y_pred_nn = np.argmax(y_pred_nn_prob, axis=1)
nn_acc = accuracy_score(y_test, y_pred_nn)
nn_f1 = f1_score(y_test, y_pred_nn, average='weighted')

print(f"Neural Network Accuracy: {nn_acc:.4f}")
print(f"Neural Network F1-Score (weighted): {nn_f1:.4f}")


TRAINING IMPROVED NEURAL NETWORK

Neural Network Architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization             │ (None, 44)             │           176 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │        23,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 9)              │           297 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 201,913 (788.72 KB)

 Trainable params: 199,905 (780.88 KB)

 Non-trainable params: 2,008 (7.84 KB)


Training Neural Network...
Neural Network Accuracy: 0.5964
Neural Network F1-Score (weighted): 0.5918


In [ ]:
# ==================== WEIGHTED VOTING ENSEMBLE ====================
print("\n" + "="*60)
print("CREATING WEIGHTED VOTING ENSEMBLE")
print("="*60)

# Get probabilities from all models
y_pred_xgb_prob = xgb_model.predict_proba(X_test_scaled)
y_pred_lgb_prob = lgb_model.predict_proba(X_test_scaled)
y_pred_cat_prob = cat_model.predict_proba(X_test_scaled)
y_pred_rf_prob = rf_model.predict_proba(X_test_scaled)

# Weighted ensemble - normalize by accuracy
total_acc = xgb_acc + lgb_acc + cat_acc + rf_acc + nn_acc
w_xgb = xgb_acc / total_acc
w_lgb = lgb_acc / total_acc
w_cat = cat_acc / total_acc
w_rf = rf_acc / total_acc
w_nn = nn_acc / total_acc

ensemble_prob = (w_xgb * y_pred_xgb_prob +
                 w_lgb * y_pred_lgb_prob +
                 w_cat * y_pred_cat_prob +
                 w_rf * y_pred_rf_prob +
                 w_nn * y_pred_nn_prob)

y_pred_ensemble = np.argmax(ensemble_prob, axis=1)
ensemble_acc = accuracy_score(y_test, y_pred_ensemble)
ensemble_f1 = f1_score(y_test, y_pred_ensemble, average='weighted')

print(f"\nEnsemble Weights:")
print(f"  XGBoost: {w_xgb:.3f}")
print(f"  LightGBM: {w_lgb:.3f}")
print(f"  CatBoost: {w_cat:.3f}")
print(f"  Random Forest: {w_rf:.3f}")
print(f"  Neural Network: {w_nn:.3f}")

print(f"\nEnsemble Accuracy: {ensemble_acc:.4f}")
print(f"Ensemble F1-Score (weighted): {ensemble_f1:.4f}")


CREATING WEIGHTED VOTING ENSEMBLE

Ensemble Weights:
  XGBoost: 0.204
  LightGBM: 0.209
  CatBoost: 0.198
  Random Forest: 0.206
  Neural Network: 0.182

Ensemble Accuracy: 0.6762
Ensemble F1-Score (weighted): 0.6739


In [ ]:
# ==================== RESULTS SUMMARY ====================
print("\n" + "="*60)
print("FINAL RESULTS SUMMARY")
print("="*60)

results = {
    'XGBoost': {'Accuracy': xgb_acc, 'F1-Score': xgb_f1},
    'LightGBM': {'Accuracy': lgb_acc, 'F1-Score': lgb_f1},
    'CatBoost': {'Accuracy': cat_acc, 'F1-Score': cat_f1},
    'Random Forest': {'Accuracy': rf_acc, 'F1-Score': rf_f1},
    'Neural Network': {'Accuracy': nn_acc, 'F1-Score': nn_f1},
    'Ensemble': {'Accuracy': ensemble_acc, 'F1-Score': ensemble_f1}
}

results_df = pd.DataFrame(results).T
print("\n", results_df)

print(f"\n🏆 BEST MODEL: {results_df['Accuracy'].idxmax()} with {results_df['Accuracy'].max():.4f} accuracy")



FINAL RESULTS SUMMARY

                 Accuracy  F1-Score
XGBoost         0.669251  0.666861
LightGBM        0.684432  0.683689
CatBoost        0.650086  0.646644
Random Forest   0.676034  0.671695
Neural Network  0.596361  0.591784
Ensemble        0.676249  0.673902

🏆 BEST MODEL: LightGBM with 0.6844 accuracy


In [ ]:
# ==================== DETAILED CLASSIFICATION REPORT ====================
print("\n" + "="*60)
print("DETAILED CLASSIFICATION REPORT - ENSEMBLE MODEL")
print("="*60)
print(classification_report(y_test, y_pred_ensemble,
                          target_names=label_encoder_target.classes_))


DETAILED CLASSIFICATION REPORT - ENSEMBLE MODEL
                                     precision    recall  f1-score   support

                        Alzheimer's       0.96      1.00      0.98      1032
                             Cancer       0.95      1.00      0.97      1032
                    Cystic fibrosis       0.56      0.51      0.53      1032
                           Diabetes       0.70      0.62      0.66      1032
                    Hemochromatosis       0.70      0.73      0.72      1032
Leber's hereditary optic neuropathy       0.85      0.87      0.86      1032
                     Leigh syndrome       0.44      0.50      0.46      1032
             Mitochondrial myopathy       0.40      0.37      0.38      1032
                          Tay-Sachs       0.50      0.50      0.50      1032

                           accuracy                           0.68      9288
                          macro avg       0.67      0.68      0.67      9288
                       we

In [ ]:
# ==================== FEATURE IMPORTANCE ====================
print("\n" + "="*60)
print("TOP 20 IMPORTANT FEATURES")
print("="*60)

feature_importance_xgb = pd.DataFrame({
    'Feature': X.columns,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nXGBoost Feature Importance (Top 20):")
print(feature_importance_xgb.head(20))


TOP 20 IMPORTANT FEATURES

XGBoost Feature Importance (Top 20):
                                         Feature  Importance
40                                Total_symptoms    0.272031
41                                Symptoms_ratio    0.140778
43                     Genetic_inheritance_score    0.050618
34                                     Symptom 5    0.033867
1                         Genes in mother's side    0.027679
33                                     Symptom 4    0.019631
30                                     Symptom 1    0.019580
2                          Inherited from father    0.019259
32                                     Symptom 3    0.018596
31                                     Symptom 2    0.017309
4                                  Paternal gene    0.017261
3                                  Maternal gene    0.017073
36                                       Age_log    0.015887
25  History of anomalies in previous pregnancies    0.015574
20        Folic acid